## Linguistic analysis of authentic and adversarial data

In [ ]:
import pandas as pd
import spacy
from transformers import pipeline
from collections import Counter

In [96]:
# read data
df_plf = pd.read_csv("../../data/authentic_data/counter_abuse_plf_annotated.csv")
df_adv = pd.read_csv("../../data/adversarial_data/responses_annotated.csv")

In [97]:
df_plf['counter_speech_processed'] = df_plf['counter_speech'].map(lambda x: x.replace("[USER]", "").replace("[PLAYER]", "").replace("[CLUB]", "").replace("[URL]", "").strip())

In [98]:
nlp = spacy.load("en_core_web_sm")
sentiment_pipeline = pipeline("sentiment-analysis")
multiclass_classifier = pipeline(task="text-classification", model="SamLowe/roberta-base-go_emotions", top_k=None)

def count_tokens(text):
    doc = nlp(text)
    return len(doc)

def most_common(lst):
    data = Counter(lst)
    if len(data) > 0:
        if len(data) > 1:
            if data.most_common(1)[0][1] == data.most_common()[1][1]:
                return [data.most_common(1)[0][0], data.most_common()[1][0]]
        return data.most_common(1)[0][0]
    else:
        return ""

def get_ne(text):
    doc = nlp(text)
    ne_list = []
    for ent in doc.ents:
        ne_list.append(ent.label_)
    return most_common(ne_list)
    
def count_pronoun(text):
    doc = nlp(text)
    cnt = 0
    for token in doc:
        if token.pos_ == 'PROPN':
            cnt += 1
    return cnt

def count_negation(text):
    doc = nlp(text)
    cnt = 0
    for token in doc:
        if token.dep_ == 'neg':
            cnt += 1
    return cnt

def sentiment_analysis(text):
    return sentiment_pipeline([text])[0]['label']

def multiclass_sentiment_analysis(text):
    model_outputs = multiclass_classifier([text])
    return model_outputs[0][0]['label']


def process_ne(ne):
    if type(ne) == str:
        if len(ne) > 0:
            return ne
        return "no entity"
    else:
        return "most common entity > 1"

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [99]:
df_adv['count_tokens'] = df_adv.apply(lambda x: count_tokens(x['context']), axis = 1)
df_adv['get_ne'] = df_adv.apply(lambda x: get_ne(x['context']), axis = 1)
df_adv['count_pronoun'] = df_adv.apply(lambda x: count_pronoun(x['context']), axis = 1)
df_adv['count_negation'] = df_adv.apply(lambda x: count_negation(x['context']), axis = 1)
df_adv['sentiment_analysis'] = df_adv.apply(lambda x: sentiment_analysis(x['context']), axis = 1)
df_adv['multiclass_sentiment_analysis'] = df_adv.apply(lambda x: multiclass_sentiment_analysis(x['context']), axis = 1)

df_plf['count_tokens'] = df_plf.apply(lambda x: count_tokens(x['counter_speech_processed']), axis = 1)
df_plf['get_ne'] = df_plf.apply(lambda x: get_ne(x['counter_speech_processed']), axis = 1)
df_plf['count_pronoun'] = df_plf.apply(lambda x: count_pronoun(x['counter_speech_processed']), axis = 1)
df_plf['count_negation'] = df_plf.apply(lambda x: count_negation(x['counter_speech_processed']), axis = 1)
df_plf['sentiment_analysis'] = df_plf.apply(lambda x: sentiment_analysis(x['counter_speech_processed']), axis = 1)
df_plf['multiclass_sentiment_analysis'] = df_plf.apply(lambda x: multiclass_sentiment_analysis(x['counter_speech_processed']), axis = 1)

df_plf['get_ne_most'] = df_plf.apply(lambda x: process_ne(x['get_ne']), axis = 1)
df_adv['get_ne_most'] = df_adv.apply(lambda x: process_ne(x['get_ne']), axis = 1)

In [107]:
df_adv.to_csv("../../data/adversarial_data/responses_annotated_linguistic_analysis.csv")
df_plf.to_csv("../../data/authentic_data/counter_abuse_plf_annotated_linguistic_analysis.csv")

In [3]:
df_adv = pd.read_csv("../../data/adversarial_data/responses_annotated_linguistic_analysis.csv")
df_plf = pd.read_csv("../../data/authentic_data/counter_abuse_plf_annotated_linguistic_analysis.csv")

In [4]:
df_plf_agree = df_plf[df_plf['label'] == 'agrees_with_the_post']
df_plf_disagree = df_plf[df_plf['label'] == 'disagrees_with_the_post']
df_plf_other = df_plf[df_plf['label'] == 'other']

In [8]:
len(df_plf_agree), len(df_plf_disagree), len(df_plf_other), 1173+2124+159

(1173, 2124, 159, 3456)

In [9]:
print(df_adv['count_tokens'].describe())
print(df_plf['count_tokens'].describe())
print(df_plf_agree['count_tokens'].describe())
print(df_plf_disagree['count_tokens'].describe())
print(df_plf_other['count_tokens'].describe())


count    1911.000000
mean       19.576138
std         7.604513
min         1.000000
25%        14.000000
50%        20.000000
75%        27.000000
max        42.000000
Name: count_tokens, dtype: float64
count    3456.000000
mean       14.612847
std        13.149986
min         1.000000
25%         6.000000
50%        10.000000
75%        19.000000
max        87.000000
Name: count_tokens, dtype: float64
count    1173.000000
mean       13.640239
std        12.199080
min         1.000000
25%         5.000000
50%        10.000000
75%        18.000000
max        63.000000
Name: count_tokens, dtype: float64
count    2124.000000
mean       15.627119
std        13.634981
min         1.000000
25%         7.000000
50%        11.000000
75%        20.000000
max        87.000000
Name: count_tokens, dtype: float64
count    159.000000
mean       8.238994
std       10.845555
min        1.000000
25%        2.000000
50%        5.000000
75%        8.000000
max       57.000000
Name: count_tokens, dtype: f

In [10]:
print(df_adv['get_ne_most'].value_counts(normalize=True).sort_index(ascending=True))
print(df_plf['get_ne_most'].value_counts(normalize=True).sort_index(ascending=True))
print('------------------')
print(df_plf_agree['get_ne_most'].value_counts(normalize=True).sort_index(ascending=True))
print(df_plf_disagree['get_ne_most'].value_counts(normalize=True).sort_index(ascending=True))
print(df_plf_other['get_ne_most'].value_counts(normalize=True).sort_index(ascending=True))

get_ne_most
CARDINAL                  0.010466
DATE                      0.010989
EVENT                     0.000523
GPE                       0.060701
LANGUAGE                  0.013605
LOC                       0.008896
NORP                      0.217164
ORDINAL                   0.009419
ORG                       0.004710
PERCENT                   0.000523
PERSON                    0.007326
QUANTITY                  0.000523
TIME                      0.001570
WORK_OF_ART               0.001047
most common entity > 1    0.051282
no entity                 0.601256
Name: proportion, dtype: float64
get_ne_most
CARDINAL                  0.039062
DATE                      0.026910
EVENT                     0.000579
FAC                       0.002025
GPE                       0.028935
LANGUAGE                  0.001736
LOC                       0.002315
MONEY                     0.002315
NORP                      0.010417
ORDINAL                   0.006366
ORG                       0.05555

In [11]:
print(df_adv['count_pronoun'].value_counts(normalize=True).sort_index(ascending=True))
print(df_plf['count_pronoun'].value_counts(normalize=True).sort_index(ascending=True))
print('------------------')
print(df_plf_agree['count_pronoun'].value_counts(normalize=True).sort_index(ascending=True))
print(df_plf_disagree['count_pronoun'].value_counts(normalize=True).sort_index(ascending=True))
print(df_plf_other['count_pronoun'].value_counts(normalize=True).sort_index(ascending=True))

count_pronoun
0     0.732077
1     0.162219
2     0.076400
3     0.019362
4     0.005233
5     0.002093
6     0.001570
7     0.000523
11    0.000523
Name: proportion, dtype: float64
count_pronoun
0     0.579572
1     0.216725
2     0.106481
3     0.048032
4     0.021123
5     0.012153
6     0.006366
7     0.001736
8     0.001447
9     0.001447
10    0.000579
11    0.002025
12    0.000289
13    0.000289
15    0.000289
16    0.000289
17    0.000289
19    0.000289
21    0.000289
23    0.000289
Name: proportion, dtype: float64
------------------
count_pronoun
0     0.620631
1     0.190963
2     0.102302
3     0.043478
4     0.021313
5     0.008525
6     0.004263
7     0.001705
8     0.000853
9     0.001705
10    0.000853
11    0.000853
12    0.000853
16    0.000853
21    0.000853
Name: proportion, dtype: float64
count_pronoun
0     0.556968
1     0.229755
2     0.109228
3     0.052260
4     0.021657
5     0.014124
6     0.007533
7     0.001412
8     0.001412
9     0.000942
10    0.000471
1

In [12]:
print(df_adv['count_negation'].value_counts(normalize=True).sort_index(ascending=True))
print(df_plf['count_negation'].value_counts(normalize=True).sort_index(ascending=True))
print('------------------')
print(df_plf_agree['count_negation'].value_counts(normalize=True).sort_index(ascending=True))
print(df_plf_disagree['count_negation'].value_counts(normalize=True).sort_index(ascending=True))
print(df_plf_other['count_negation'].value_counts(normalize=True).sort_index(ascending=True))

count_negation
0    0.652538
1    0.264783
2    0.068550
3    0.010989
4    0.002616
5    0.000523
Name: proportion, dtype: float64
count_negation
0    0.774016
1    0.190972
2    0.028935
3    0.004340
4    0.001157
5    0.000579
Name: proportion, dtype: float64
------------------
count_negation
0    0.796249
1    0.177323
2    0.023018
3    0.002558
4    0.000853
Name: proportion, dtype: float64
count_negation
0    0.748588
1    0.209981
2    0.033427
3    0.005650
4    0.001412
5    0.000942
Name: proportion, dtype: float64
count_negation
0    0.949686
1    0.037736
2    0.012579
Name: proportion, dtype: float64


In [13]:
print(df_adv['sentiment_analysis'].value_counts(normalize=True).sort_index(ascending=True))
print(df_plf['sentiment_analysis'].value_counts(normalize=True).sort_index(ascending=True))
print('------------------')
print(df_plf_agree['sentiment_analysis'].value_counts(normalize=True).sort_index(ascending=True))
print(df_plf_disagree['sentiment_analysis'].value_counts(normalize=True).sort_index(ascending=True))
print(df_plf_other['sentiment_analysis'].value_counts(normalize=True).sort_index(ascending=True))

sentiment_analysis
NEGATIVE    0.77551
POSITIVE    0.22449
Name: proportion, dtype: float64
sentiment_analysis
NEGATIVE    0.697338
POSITIVE    0.302662
Name: proportion, dtype: float64
------------------
sentiment_analysis
NEGATIVE    0.707587
POSITIVE    0.292413
Name: proportion, dtype: float64
sentiment_analysis
NEGATIVE    0.698211
POSITIVE    0.301789
Name: proportion, dtype: float64
sentiment_analysis
NEGATIVE    0.610063
POSITIVE    0.389937
Name: proportion, dtype: float64


In [14]:
print(df_adv['multiclass_sentiment_analysis'].value_counts(normalize=True).sort_index(ascending=True))
print(df_plf['multiclass_sentiment_analysis'].value_counts(normalize=True).sort_index(ascending=True))
print('------------------')
print(df_plf_agree['multiclass_sentiment_analysis'].value_counts(normalize=True).sort_index(ascending=True))
print(df_plf_disagree['multiclass_sentiment_analysis'].value_counts(normalize=True).sort_index(ascending=True))
print(df_plf_other['multiclass_sentiment_analysis'].value_counts(normalize=True).sort_index(ascending=True))

multiclass_sentiment_analysis
admiration        0.037677
amusement         0.007849
anger             0.025118
annoyance         0.045003
approval          0.050759
caring            0.016745
confusion         0.009419
curiosity         0.022501
desire            0.015699
disappointment    0.024071
disapproval       0.085819
disgust           0.013605
embarrassment     0.001047
excitement        0.001570
fear              0.009419
gratitude         0.004186
joy               0.005756
love              0.002616
nervousness       0.001570
neutral           0.593407
optimism          0.004710
pride             0.001570
realization       0.007326
remorse           0.001570
sadness           0.009419
surprise          0.001570
Name: proportion, dtype: float64
multiclass_sentiment_analysis
admiration        0.042245
amusement         0.040509
anger             0.075231
annoyance         0.066262
approval          0.035880
caring            0.015914
confusion         0.010417
curiosity       

In [16]:
df_plf_disagree['get_ne_most'].value_counts(normalize=True).sort_index(ascending=True).to_csv("plf_named_entity_most_disagree.csv")
df_plf_agree['get_ne_most'].value_counts(normalize=True).sort_index(ascending=True).to_csv("plf_named_entity_most_agree.csv")
df_plf_other['get_ne_most'].value_counts(normalize=True).sort_index(ascending=True).to_csv("plf_named_entity_most_other.csv")
df_plf_disagree['multiclass_sentiment_analysis'].value_counts(normalize=True).sort_index(ascending=True).to_csv("plf_multiclass_sentiment_analysis_disagree.csv")
df_plf_agree['multiclass_sentiment_analysis'].value_counts(normalize=True).sort_index(ascending=True).to_csv("plf_multiclass_sentiment_analysis_agree.csv")
df_plf_other['multiclass_sentiment_analysis'].value_counts(normalize=True).sort_index(ascending=True).to_csv("plf_multiclass_sentiment_analysis_other.csv")

In [15]:
df_plf_disagree['get_ne_most'].value_counts().sort_index(ascending=True).to_csv("plf_named_entity_cnt_most_disagree.csv")
df_plf_agree['get_ne_most'].value_counts().sort_index(ascending=True).to_csv("plf_named_entity_cnt_most_agree.csv")
df_plf_other['get_ne_most'].value_counts().sort_index(ascending=True).to_csv("plf_named_entity_cnt_most_other.csv")